In [1]:
import pandas as pd
from dnn.data.dataloader import init_embed_loaders
from dnn.parser import get_sv_parser

ModuleNotFoundError: No module named 'dnn'

In [23]:
def secToSample(sec):
    return int(16000 * sec)

options = get_sv_parser().parse_args(args=[])
options.n_dct_filters = 40
options.n_mels = 40
options.timeshift_ms = 100
options.data_folder = "/home/muncok/DL/dataset/SV_sets"
options.window_size= 0.025
options.window_stride= 0.010
options.cache_size = 32768

options.input_format = "fbank"
options.input_clip = True
options.input_length = secToSample(3) # if input_clip is false, it doesn't affect anything
options.splice_frames = secToSample(0.1)//160+1

options.data_folder = "/home/muncok/DL/dataset/SV_sets/reddots_r2015q4_v1/wav/"

In [24]:
m_part1_ndx = pd.read_pickle("../interspeech2018/trials/reddots/m_part1/m_part1_ndx.pkl")
m_part1_trn = pd.read_pickle('../interspeech2018/trials/reddots/m_part1/m_part1_trn.pkl')

In [25]:
trn = m_part1_trn
ndx = m_part1_ndx

In [26]:
val_dataloader = init_embed_loaders(options, trn) 
trn_embeddings, _ = embeds(options, val_dataloader, model, lda)

embed_dim = trn_embeddings.shape[-1]
trn_id = list(trn.id.unique())
spk_model_dict = {}
for id in trn_id:
    index = np.nonzero(trn.id == id)
    spk_model_dict[id] = trn_embeddings[index].mean(0, True)

all_trials = ndx.id.unique().tolist()
scores = dict()
for t in range(4):
    scores[t] = []

AttributeError: 'Series' object has no attribute 'sent'

In [27]:
trial_id = all_trials[99]
print(trial_id)
trial_ndx = ndx[(ndx.id == trial_id)].reset_index()
trial_embed_idx = np.nonzero(m_part1_files.file.isin(trial_ndx.file))
trial_embeds = ndx_embeddings[trial_embed_idx]
sim = F.cosine_similarity(trial_embeds, spk_model_dict[trial_id])
for t in range(4):
    trial_type_idx = trial_ndx[trial_ndx.trial_type == t].index.tolist()
    scores[t].append(sim[trial_type_idx])
        
# [TC, TW, IC, IW]
for t in range(4):
    scores[t] = torch.cat(scores[t])

from sklearn.metrics import roc_curve

for t in range(1,4):
    score_vector = np.concatenate((scores[0], scores[t]))
    label_vector = np.concatenate((np.ones(len(scores[0])), 
                               np.zeros(len(scores[t]))))
    fpr, tpr, thres = roc_curve(label_vector, score_vector, pos_label=1)
    eer = fpr[np.nanargmin(np.abs(fpr - (1 - tpr)))]
    thres = thres[np.nanargmin(np.abs(fpr - (1 - tpr)))]
    print("[{}] eer: {:.2f}, thres: {:.5f}".format(err_type[t], eer, thres))

enroll_df = trn[trn.id == trial_id] 
enroll_df['trial_type'] = -1

# row scores in pos_test
t_type = 0
n_test = 5
hard_pos_test = np.argsort(scores[t_type])[:n_test]
pos_test_df = trial_ndx[trial_ndx.trial_type == t_type].iloc[hard_pos_test]

t_type = 2
hard_neg_test = np.argsort(scores[t_type])[-n_test:]
neg_test_df =trial_ndx[trial_ndx.trial_type == t_type].iloc[hard_neg_test]

from shutil import copyfile
trial_folder = "trials/human_perf/reddots/{}".format(trial_id)
if not os.path.isdir(trial_folder):
    os.makedirs(trial_folder)

test_df = pd.concat([pos_test_df, neg_test_df])
perm_idxs = np.random.permutation(np.arange(len(test_df)))

i = 0
for idx, row in enroll_df.iterrows():
    copyfile(os.path.join(options.data_folder, row.file), 
             os.path.join(trial_folder, "e_{}.wav".format(i)))
    i+=1

i = 0
random_idx = np.random.permutation(np.arange(len(test_df)))
save_file_names = []
for _, row in test_df.iterrows():
    save_file_names.append("t_{}.wav".format(perm_idxs[i]))
    copyfile(os.path.join(options.data_folder, row.file), 
             os.path.join(trial_folder, save_file_names[-1]))
    i+=1

test_df['to_file'] = save_file_names

with open(os.path.join(trial_folder, "test_labels.txt"), "w") as f:
    for _, row in test_df.iterrows():
        if row.trial_type == 0:
            f.write("{}\t{}\n".format(row.to_file, "1"))
        else:
            f.write("{}}\t{}\n".format(row_to_file, "0"))

test_df.to_pickle(os.path.join(trial_folder, "test_audios.pkl"))

NameError: name 'all_trials' is not defined